In [0]:
use catalog identifier(:catalog);
use schema identifier(:schema);

In [0]:
declare or replace qry_str string;

set var qry_str = 
  "create or replace view metv_query_history
  (
    Date comment 'Date when Databricks received the request.',
    `Workspace Name` comment 'The name of the workspace.',
    `Client Application` comment 'Client application that ran the statement. For example: Databricks SQL, Tableau, and Power BI. Nulls and blanks are labled as Unknown',
    `Compute Type` comment 'The type of compute (Warehouse, Serverless, etc.)',
    `Warehouse Name` comment 'The name of the SQL warehouse.',
    `Warehouse Type` comment 'The type of SQL warehouse. Possible values are CLASSIC, PRO, and SERVERLESS.',
    `Warehouse Size` comment 'The cluster size of the SQL warehouse. Possible values are 2X_SMALL, X_SMALL, SMALL, MEDIUM, LARGE, X_LARGE, 2X_LARGE, 3X_LARGE, and 4X_LARGE.',
    `Query Count` comment 'Count of queries.',
    `Query Duration Sec P50` comment 'P50 of total execution time of the statement in seconds (Excluding result fetch time).',
    `Query Duration Sec P90` comment 'P90 of total execution time of the statement in seconds (Excluding result fetch time).',
    `Query Duration Sec P95` comment 'P95 of total execution time of the statement in seconds (Excluding result fetch time).'
  )
  with metrics
  language yaml
  as $$
  version: 0.1
  
  source: " || :catalog || '.' || :schema || ".fct_query_history
  
  joins:
    - name: calendar
      source: " || :catalog || '.' || :schema || ".dim_calendar
      on: calendar.calendar_key = source.calendar_key

    - name: compute
      source: " || :catalog || '.' || :schema || ".dim_compute
      on: compute.compute_key = source.compute_key

    - name: workspace
      source: " || :catalog || '.' || :schema || ".dim_workspace
      on: workspace.workspace_key = source.workspace_key
  
  dimensions:
    - name: Date
      expr: calendar.calendar_date
    - name: Workspace Name
      expr: workspace.workspace_name
    - name: Client Application
      expr: source.client_application
    - name: Compute Type
      expr: compute.compute_type
    - name: Warehouse Name
      expr: compute.warehouse_name
    - name: Warehouse Type
      expr: compute.warehouse_type
    - name: Warehouse Size
      expr: compute.warehouse_size

  measures:
    - name: Query Count
      expr: count(statement_id)
    - name: Query Duration Sec P50
      expr: percentile(total_duration_ms / 1000, 0.5)
    - name: Query Duration Sec P90
      expr: percentile(total_duration_ms / 1000, 0.9)
    - name: Query Duration Sec P95
      expr: percentile(total_duration_ms / 1000, 0.95)
  $$";

execute immediate qry_str;